In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.utils import shuffle
from sklearn.linear_model import Ridge
from numpy import nan as na
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [2]:
df = pd.read_csv('Train.csv')

FEATURE GENERATION

In [3]:
def generate_features(df):
    df['percent_change_24h']/=100
    df['p24'] = df.open * ((df.percent_change_24h/24)+1) # prediction using 24 hour value
    df.market_cap/=1000000000
    df.market_cap_global/=1000000000
    df['mid'] = (df['high'] * 0.5) + (df['low']* 0.5) 
    df['mo'] = df.mid / df.open # MO IS MID DIV BY OPEN
    df['ho'] = df.high / df.open
    df['mr'] = df.market_cap / df.market_cap_global # MR IS MARKET RATIO
    df['meanmo'] =  df[['mid','open']].mean(axis=1) # MEAN OF MID AND OPEN
    df['predmr'] = df.open * df.mr
    df['predmeanmo'] = df.open * df.meanmo
    df['e'] = (df.mid>df.open).astype('int') # MID GREATER THAN OPEN?
    df.fillna(0, inplace=True)
    return df

In [4]:
train_df = generate_features(df)
train_df = train_df[train_df['close'].notna()]
train_df = train_df[train_df.close <= train_df.high]
train_df = train_df[train_df.close >= train_df.low]

USE MORE UNIQUE FEATURES TO TRAIN MODEL

In [5]:
cols =['open', 'predmeanmo', 'market_cap_global', 'e',
       'percent_change_24h' ,'p24' ,'ho' ,'mo' ,'mid',
       'market_cap', 'predmr' ,'mr'] 

CREATE POLYNOMIAL FEATURES

In [6]:
pf = PolynomialFeatures(degree = 2, include_bias = False, interaction_only = False)

SCALE FEATURES

In [7]:
sc = StandardScaler()

FIT MODEL TO WHOLE TRAIN SET

In [8]:
model = Ridge(alpha=0, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)
model.fit(sc.fit_transform(pf.fit_transform(train_df[cols])),train_df.close)                    

C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


Ridge(alpha=0, normalize=False)

In [9]:
model

Ridge(alpha=0, normalize=False)

In [10]:
import pickle
pickle_out = open("predictor.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [11]:
def clip_prediction(test_df):
    test_df.pred[(test_df.pred<test_df.low)] = test_df.low
    test_df.pred[(test_df.pred>test_df.high)] = test_df.high
    test_df.pred[(test_df.open<=0)&(test_df.high<=0)&(test_df.low<=0)] = 0
    return test_df

In [13]:
import joblib
from joblib import dump

In [14]:
dump(sc, 'std_scaler.bin', compress=True)

['std_scaler.bin']

In [15]:
dump(pf, 'poly_features.bin', compress=True)

['poly_features.bin']

In [17]:
from joblib import load
sc=load('std_scaler.bin')
pf=load('poly_features.bin')
